In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

## What is RAG (retrieval augmented generation)?
Basically, shoving lot of extra information in the prompt.

In [2]:
# Example 

from langchain.chat_models import init_chat_model
model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

prompt_template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""

response = model.invoke(
    prompt_template.format(
        context="Ajit has two sisters, Preeti and Sweta. Ajit is male.",
        question="Who is Preeti's bother?"         
    ))

print(response)

/Users/nithurshen/SNU/MAT496/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


content="Ajit is Preeti's brother. He is also the brother of Sweta. There is no other information about any other brother of Preeti." additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 119, 'total_tokens': 152, 'completion_time': 0.122211098, 'prompt_time': 0.01990158, 'queue_time': 0.05437354, 'total_time': 0.142112678}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--7c00d253-d3f2-41a8-b4e0-5f2776a15b5b-0' usage_metadata={'input_tokens': 119, 'output_tokens': 33, 'total_tokens': 152}


In [3]:
print(response.content)

Ajit is Preeti's brother. He is also the brother of Sweta. There is no other information about any other brother of Preeti.


>RAG is all about cleverly pushing is as much information in the context with minimum possible tokens

In [4]:
# little non-trival RAG example

# grab a novel
file = open("indianTales.txt")
text = file.read()

In [5]:
# lets try to talk with this book

from langchain.chat_models import init_chat_model
model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

# Prepare your prompt
prompt_template = """You are a novel reader. You are given collection of stories:
{collection_of_stories}
You are tasked to make a list of story titles in this collection. Write a short summary for each story in Hindi Language. Skip the story from the list, if the story is not provided in the text. 
"""

response = model.invoke(prompt_template.format(collection_of_stories = text[:len(text)//20]))

print(response)

content='यहाँ दी गई कहानियों की सूची है जिनके साथ एक छोटी सारांश है हिंदी में:\n\n\n1. **शेर और बगुला** - इस कहानी में, एक शेर के गले में एक हड्डी फंस जाती है और वह बहुत दर्द से पीड़ित होता है। एक बगुला उसे बचाने के लिए आता है और हड्डी को निकाल देता है, लेकिन शेर उसे धन्यवाद नहीं देता और कहता है कि वह उसे पहले ही मार सकता था जब वह उसके मुंह में था।\n\n2. **राजकुमार और राजकुमारी लबाम** - इस कहानी में, एक राजकुमार अपनी माँ की आज्ञा के विपरीत जाकर एक जंगल में जाता है जहाँ वह एक राजकुमारी लबाम के बारे में सुनता है। वह उसे ढूंढने का फैसला करता है और कई चुनौतियों का सामना करते हुए, वह अंततः उसे पा लेता है।\n\n\nक्योंकि अन्य कहानियाँ पूरी नहीं दी गई हैं, इसलिए मैं उनके लिए सारांश नहीं लिख सकता।' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 348, 'prompt_tokens': 5232, 'total_tokens': 5580, 'completion_time': 1.255214408, 'prompt_time': 0.404267379, 'queue_time': 0.05598511, 'total_time': 1.659481787}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint'

In [6]:
# Better printing

from IPython.display import Markdown
Markdown(response.content)

यहाँ दी गई कहानियों की सूची है जिनके साथ एक छोटी सारांश है हिंदी में:


1. **शेर और बगुला** - इस कहानी में, एक शेर के गले में एक हड्डी फंस जाती है और वह बहुत दर्द से पीड़ित होता है। एक बगुला उसे बचाने के लिए आता है और हड्डी को निकाल देता है, लेकिन शेर उसे धन्यवाद नहीं देता और कहता है कि वह उसे पहले ही मार सकता था जब वह उसके मुंह में था।

2. **राजकुमार और राजकुमारी लबाम** - इस कहानी में, एक राजकुमार अपनी माँ की आज्ञा के विपरीत जाकर एक जंगल में जाता है जहाँ वह एक राजकुमारी लबाम के बारे में सुनता है। वह उसे ढूंढने का फैसला करता है और कई चुनौतियों का सामना करते हुए, वह अंततः उसे पा लेता है।


क्योंकि अन्य कहानियाँ पूरी नहीं दी गई हैं, इसलिए मैं उनके लिए सारांश नहीं लिख सकता।

### Problem: Too much raw information in the context makes the prompt too long.
- Costly
- adds noise
### Solution: Use RAG
https://python.langchain.com/docs/tutorials/rag/
### To understand RAG, we need to understand Semantic Search
https://python.langchain.com/docs/tutorials/retrievers/

In [8]:
# load text using RELEVANT loader
from langchain_community.document_loaders import TextLoader
loader = TextLoader("indianTales.txt")
docs = loader.load()

In [9]:
# Split document into small chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split given book into {len(all_splits)} sub-documents.")

Split given book into 563 sub-documents.


### Embedding

In [10]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Create a vector store
# (CLASSROOM DISCUSSION: What are vector stores? What do we make them?)
from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embeddings)

# Adding documents to vector store
document_ids = vector_store.add_documents(documents=all_splits)


DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [15]:
# document_ids

In [18]:
# extract chunks which matches with your query

search_results = vector_store.similarity_search_with_score(
    "What is the role of Lion in the story?",
    k = 10
)

In [19]:
search_results

[(Document(id='6745ea9b-12c8-45d5-878a-03c21ec617c9', metadata={'source': 'indianTales.txt', 'start_index': 341149}, page_content="In the following Notes I give, as on the two previous occasions, the\n_source_ whence I derived the tale, then _parallels_, and finally\n_remarks_. For Indian _parallels_ I have been able to refer to Major\nTemple's remarkable Analysis of Indian Folk-tale incidents at the end\nof _Wideawake Stories_ (pp. 386-436), for European ones to my\nalphabetical List of Incidents, with bibliographical references, in\n_Transactions of Folk-Lore Congress_, 1892, pp. 87-98. My _remarks_\nhave been mainly devoted to tracing the relation between the Indian\nand the European tales, with the object of showing that the latter\nhave been derived from the former. I have, however, to some extent\nhandicapped myself, as I have avoided giving again the Indian versions\nof stories already given in _English Fairy Tales_ or _Celtic Fairy\nTales_.\n\n\nI. THE LION AND THE CRANE."),
  

### What is RAG?
Retrieve using semantic search and dump the similar chunks in the context of the prompt.
LLM sees the question and retrieved docs in its prompt and generates tokens accordingly.

In [ ]:
prompt_template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""

In [20]:
doc_content = "\n\n".join(doc.page_content+"\n"+"="*50+"\n" for (doc,score) in search_results)
print(doc_content)

In the following Notes I give, as on the two previous occasions, the
_source_ whence I derived the tale, then _parallels_, and finally
_remarks_. For Indian _parallels_ I have been able to refer to Major
Temple's remarkable Analysis of Indian Folk-tale incidents at the end
of _Wideawake Stories_ (pp. 386-436), for European ones to my
alphabetical List of Incidents, with bibliographical references, in
_Transactions of Folk-Lore Congress_, 1892, pp. 87-98. My _remarks_
have been mainly devoted to tracing the relation between the Indian
and the European tales, with the object of showing that the latter
have been derived from the former. I have, however, to some extent
handicapped myself, as I have avoided giving again the Indian versions
of stories already given in _English Fairy Tales_ or _Celtic Fairy
Tales_.


I. THE LION AND THE CRANE.


mouth struck one end of the bone with his beak. Whereupon the bone
dropped and fell out. As soon as he had caused the bone to fall, he
got out of the

In [21]:
# make the LLM read see the prompt, and analyse the retrieved document, and generate response

from langchain.chat_models import init_chat_model
model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

prompt_template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""

response = model.invoke(prompt_template.format(
    context=doc_content,
    question="What is the role of Lion in the story?"))

In [22]:
# Better printing

from IPython.display import Markdown
Markdown(response.content)

The lion in the story is portrayed as an ungrateful character who fails to show appreciation for the crane's help in removing a bone from its mouth. The lion's reply to the crane's request for reward is that it's lucky the crane is still alive after being in its mouth. The lion is also compared to Devadatta the Traitor, implying that it represents a treacherous or untrustworthy figure.

# RAG Summary

In [23]:
# RAG summary

# Read a doc
from langchain_community.document_loaders import TextLoader
loader = TextLoader("indianTales.txt")
docs = loader.load()

# Split document into small chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split given book into {len(all_splits)} sub-documents.")

# embedding
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Create a vector store
from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embeddings)

# Adding documents to vector store
document_ids = vector_store.add_documents(documents=all_splits)


Split given book into 563 sub-documents.


In [24]:
# extract chunks which matches with your query

search_results = vector_store.similarity_search_with_score(
    "What is the role of Lion in the story?",
    k = 10
)

doc_content = "\n\n".join(doc.page_content+"\n"+"="*50+"\n" for (doc,score) in search_results)
print(doc_content)

In the following Notes I give, as on the two previous occasions, the
_source_ whence I derived the tale, then _parallels_, and finally
_remarks_. For Indian _parallels_ I have been able to refer to Major
Temple's remarkable Analysis of Indian Folk-tale incidents at the end
of _Wideawake Stories_ (pp. 386-436), for European ones to my
alphabetical List of Incidents, with bibliographical references, in
_Transactions of Folk-Lore Congress_, 1892, pp. 87-98. My _remarks_
have been mainly devoted to tracing the relation between the Indian
and the European tales, with the object of showing that the latter
have been derived from the former. I have, however, to some extent
handicapped myself, as I have avoided giving again the Indian versions
of stories already given in _English Fairy Tales_ or _Celtic Fairy
Tales_.


I. THE LION AND THE CRANE.


mouth struck one end of the bone with his beak. Whereupon the bone
dropped and fell out. As soon as he had caused the bone to fall, he
got out of the

In [25]:
prompt_template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""

response = model.invoke(prompt_template.format(
    context=doc_content,
    question="What is the role of Lion in the story?"))


from IPython.display import Markdown
Markdown(response.content)

The lion in the story is portrayed as ungrateful and selfish, as it fails to show appreciation to the crane for helping it by removing a bone from its mouth. The lion's response to the crane's expectation of gratitude is that it's lucky to be alive after being between its teeth. The lion is also symbolic, as it is mentioned that when the great Teacher, Gautama the Buddha, told this tale, he used to add that the lion was Devadatta the Traitor.